In [14]:
#  Question 4 (Your Task)
# You need to process a list of images using ProcessPoolExecutor.
# Each image processing takes random time between 3 to 6 seconds.
# Requirements:

# If an image takes more than 4 seconds, cancel it or log it as "Too Slow."

# Process the rest normally and log the results.

# 👉 Hints:
# You will need to track start times.

# You can explore wait() function from concurrent.futures.

# Timeout handling is key.
import random 
from concurrent.futures import ThreadPoolExecutor,as_completed
import time
start=time.perf_counter()
end=time.perf_counter()
def imageprocess(img):
    duration = random.uniform(2, 6)
    if duration > 4:
        raise Exception(f"Processing took too long: {duration:.2f} seconds")
    time.sleep(duration)
    return "image processed successfully"
    
images=['img1','img2','img3','img4']
success=[]
timeout=[]
with ThreadPoolExecutor() as executor:
    futures={executor.submit(imageprocess,img):img for img in images}
    for future in as_completed(futures):
        try:
            success.append((futures[future],future.result()))
        except Exception :
            timeout.append((futures[future],"Time increase 4 second"))
            #future.cancel()  only work on tasks which are not execued
print("summary")
print("Sucessfull tasks : ", success)
print("Unsucessfull Tasks : ",timeout)

summary
Sucessfull tasks :  [('img3', 'image processed successfully'), ('img2', 'image processed successfully')]
Unsucessfull Tasks :  [('img4', 'Time increase 4 second'), ('img1', 'Time increase 4 second')]


In [13]:
!python processor.py


img3 processing for 3.11 seconds
img1 processing for 3.43 seconds
img4 processing for 4.19 seconds
img2 processing for 5.30 seconds

Summary:
Successful tasks:  [('img3', 'img3 processed successfully in 3.11 seconds'), ('img1', 'img1 processed successfully in 3.43 seconds')]
Unsuccessful tasks:  [('img4', 'img4 processing took too long: 4.19 seconds'), ('img2', 'img2 processing took too long: 5.30 seconds')]


In [5]:
# Problem:
# You are downloading a list of files from multiple URLs using ThreadPoolExecutor.
# Some files may fail to download (raise exceptions).
# Write a program that:

# Continues downloading all files even if some fail.

# Collects both successful results and errors with proper logging.

# Uses as_completed.
 from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import random

def download_file(url):
    print(f"Downloading from {url}")
    time.sleep(random.uniform(0.5, 2))  # Simulate variable download times
    if random.random() < 0.3:  # 30% chance to fail
        raise Exception(f"Failed to download {url}")
    return f"Downloaded from {url}"

urls = ['file1', 'file2', 'file3', 'file4', 'file5']

successes = []
failures = []

with ThreadPoolExecutor() as executor:
    futures = {executor.submit(download_file, url): url for url in urls}
    
    for future in as_completed(futures):
        url = futures[future]
        try:
            result = future.result()
            successes.append(result)
            print(f"Success: {result}")
        except Exception as e:
            failures.append((url, str(e)))
            print(f"Error: {e}")

print("\nSummary:")
print("Successes:", successes)
print("Failures:", failures)
print(futures.values())

Success: Downloaded from file3
Error: Failed to download file1
Success: Downloaded from file2
Success: Downloaded from file4
Error: Failed to download file5

Summary:
Successes: ['Downloaded from file3', 'Downloaded from file2', 'Downloaded from file4']
Failures: [('file1', 'Failed to download file1'), ('file5', 'Failed to download file5')]
dict_values(['file1', 'file2', 'file3', 'file4', 'file5'])


In [1]:
# Question 2:
# Problem:
# You have a CPU-bound task (factorial calculation).
# You need to:

# Process a list of large numbers in parallel using ProcessPoolExecutor.

# Retrieve results as they complete.

# Ensure the fastest task is processed first, even if it was submitted last.

from concurrent.futures import ProcessPoolExecutor, as_completed
import time
import math

def compute_factorial(n):
    time.sleep(2)  # Simulate heavy CPU work
    return math.factorial(n)

numbers = [100, 50, 70, 30]

with ProcessPoolExecutor() as executor:
    futures = {executor.submit(compute_factorial, num): num for num in numbers}

    for future in as_completed(futures):
        number = futures[future]
        try:
            result = future.result()
            print(f"Factorial of {number} computed successfully.")
        except Exception as e:
            print(f"Error processing {number}: {e}")


Error processing 100: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing 50: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing 70: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing 30: A process in the process pool was terminated abruptly while the future was running or pending.


In [9]:
#make .py file of this to run it succesfully
from concurrent.futures import ProcessPoolExecutor, as_completed
import time

def heavy_cpu_task(n):
    # Simulate CPU work: sum of squares up to n
    result = sum(i * i for i in range(n))
    return f"Sum of squares up to {n} is {result}"

numbers = [10, 20, 30, 40]  # Large but manageable

with ProcessPoolExecutor() as executor:
    futures = {executor.submit(heavy_cpu_task, num): num for num in numbers}

    for future in as_completed(futures):
        number = futures[future]
        try:
            result = future.result()
            print(f"Result for {number}: {result}")
        except Exception as e:
            print(f"Error processing {number}: {e}")


Error processing 10: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing 20: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing 30: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing 40: A process in the process pool was terminated abruptly while the future was running or pending.


In [3]:
# Problem:
# Use ThreadPoolExecutor to scrape a list of websites concurrently.
# Requirements:

# If total scraping takes more than 10 seconds, terminate remaining tasks.

# Report which websites completed and which were cancelled.
from concurrent.futures import ThreadPoolExecutor, as_completed, TimeoutError
import time
import random

def scrape_site(site):
    print(f"Scraping {site}")
    time.sleep(random.uniform(2, 5))  # Simulate variable scraping times
    return f"Scraped data from {site}"

sites = ['site1', 'site2', 'site3', 'site4', 'site5']

completed_sites = []
cancelled_sites = []

with ThreadPoolExecutor() as executor:
    futures = {executor.submit(scrape_site, site): site for site in sites}

    try:
        for future in as_completed(futures, timeout=10):
            site = futures[future]
            try:
                result = future.result()
                completed_sites.append(site)
                print(result)
            except Exception as e:
                print(f"Error scraping {site}: {e}")
    except TimeoutError:
        print("\nTimeout reached. Cancelling remaining tasks...")

    for future in futures:
        if not future.done():
            future.cancel()
            cancelled_sites.append(futures[future])

print("\nSummary:")
print("Completed Sites:", completed_sites)
print("Cancelled Sites:", cancelled_sites)


Scraping site1
Scraping site2
Scraping site3
Scraping site4
Scraping site5
Scraped data from site4
Scraped data from site2
Scraped data from site5
Scraped data from site1
Scraped data from site3

Summary:
Completed Sites: ['site4', 'site2', 'site5', 'site1', 'site3']
Cancelled Sites: []


In [ ]:
# ❓ Question 5 (Your Task)
# You are given two lists:
# One list of I/O-bound tasks (web scraping URLs).
# One list of CPU-bound tasks (data processing jobs).
# Requirements:
# Run I/O-bound tasks using ThreadPoolExecutor.
# Run CPU-bound tasks using ProcessPoolExecutor.
# Ensure both sets of tasks run concurrently.
# Collect all results together.
# 👉 Hints:
# You will need to start both executors at the same time.
# You can use as_completed() on both sets of futures together.
# Think about using multiple with blocks or separate executor.submit calls❓ Question 5 (Your Task)
# You are given two lists:
# One list of I/O-bound tasks (web scraping URLs).
# One list of CPU-bound tasks (data processing jobs).

# Requirements:

# Run I/O-bound tasks using ThreadPoolExecutor.
# Run CPU-bound tasks using ProcessPoolExecutor.
# Ensure both sets of tasks run concurrently.
# Collect all results together.

# 👉 Hints:
# You will need to start both executors at the same time.
# You can use as_completed() on both sets of futures together.
# Think about using multiple with blocks or separate executor.submit calls      
#.py file
import time
import random
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed

def fetch_data(url):
    print(f"Fetching from {url}")
    time.sleep(random.uniform(1, 3))  # Simulating network delay
    return f"Data from {url}"

def process_data(data):
    print(f"Processing {data}")
    time.sleep(random.uniform(2, 4))  # Simulating CPU-heavy task
    result = sum(i * i for i in range(1000000))  # Simulated CPU load
    return f"Processed {data}"

if __name__ == "__main__":
    urls = ['site1.com', 'site2.com', 'site3.com', 'site4.com']
    datasets = ['dataset1', 'dataset2', 'dataset3', 'dataset4']

    all_futures = {}

    with ThreadPoolExecutor() as thread_executor, ProcessPoolExecutor() as process_executor:
        # Submit I/O tasks
        for url in urls:
            future = thread_executor.submit(fetch_data, url)
            all_futures[future] = url

        # Submit CPU tasks
        for data in datasets:
            future = process_executor.submit(process_data, data)
            all_futures[future] = data

        # Process results as they complete
        for future in as_completed(all_futures):
            task = all_futures[future]
            try:
                result = future.result()
                print(f"Task {task} finished with result: {result}")
            except Exception as e:
                print(f"Task {task} raised an exception: {e}")

    print("\nAll tasks completed.")
